# 引进包
在这个新的baseline里面，我们需要引进tqdm, sklearn KFold, lightgbm

In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import math


from sklearn.metrics import mean_squared_error,explained_variance_score
from sklearn.model_selection import KFold
import lightgbm as lgb

import os
import glob
import warnings
warnings.filterwarnings('ignore')

# 读取数据
- 读取清洗过的事件和码头数据

In [48]:
# baseline只用到gps定位数据，即train_gps_path
train_gps_path = 'data/train0711.csv'
test_data_path = 'data/R2_ATest0711.csv'
order_data_path = 'data/event_fixed_0612.csv'
port_data_path = 'data/port_fixed_0612.csv'

In [ ]:
# 取前1000000行
debug = True
NDATA = 2000000
i = 0

# reader = pd.read_csv(train_gps_path,chunksize=NDATA,header=None,names=column_list,iterator=True)

In [ ]:
# directory = 'F:/grad/HWC/Chusai/'
# directory = '/run/media/shxiao/Academic/grad/HWC/ChuSai/'
inner_directory = directory+'data/traindata/'
if np.shape(glob.glob(inner_directory))[0] == 0:
    os.mkdir(inner_directory)
    print('the directory',inner_directory,'is made')

In [50]:
def get_data(data, mode = 'train'):
    assert mode=='train' or mode=='test'
    
    # data['timestamp'] = pd.to_datetime(data['timestamp'], infer_datetime_format=True)
    data['longitude'] = data['longitude'].astype(float)
    data['loadingOrder'] = data['loadingOrder'].astype(str)
    data['latitude'] = data['latitude'].astype(float)
    data['speed'] = data['speed'].astype(float)
    data['direction'] = data['direction'].astype(float)

    data["TRANSPORT_TRACE"] = data['TRANSPORT_TRACE'].astype(str)
    # data["Start"] = data["TRANSPORT_TRACE"].apply(lambda x: x.split("-")[0])
    if mode=='train':
        data['vesselNextportETA'] = pd.to_datetime(data['vesselNextportETA'], infer_datetime_format=True)
        # data["End"] = data["TRANSPORT_TRACE"].apply(lambda x: x.split("-")[-1])
        # data['StartEnd'] = data['Start'].values+'-'+data["End"].values
    elif mode=='test':
        data['temp_timestamp'] = data['timestamp']
        data['onboardDate'] = pd.to_datetime(data['onboardDate'], infer_datetime_format=True)

    print('========== Finish Collecting Data ==========')
    return data

# 跑代码
## 读取测试集

In [95]:
test_data = pd.read_csv('F:/grad/HWC/ChuSai/'+test_data_path)
course_list = list(test_data["TRANSPORT_TRACE"].unique())
test_data["Start"] = test_data["TRANSPORT_TRACE"].apply(lambda x: x.split("-")[0])
test_data["End"] = test_data["TRANSPORT_TRACE"].apply(lambda x: x.split("-")[-1])
start_list = list(test_data["Start"].unique())
start_list.append('SGSIN')
end_list = list(test_data["End"].unique())
end_list[-2] = 'ARBUE'
timestamp = test_data['timestamp']
test_data = get_data(test_data, mode='test')

========== Finish Collecting Data ==========


In [96]:
# 寻找路线种类数
order_list = test_data['loadingOrder'].unique()
order_length = []
for i in range(len(order_list)):
    dff = test_data[test_data['loadingOrder'] == order_list[i]].shape[0]
    order_length.append(dff)

In [53]:
order_length

[43,
 100,
 292,
 160,
 104,
 67,
 562,
 1914,
 1692,
 877,
 178,
 156,
 38,
 402,
 136,
 25,
 98,
 220,
 330,
 150,
 2716,
 147,
 190,
 842,
 4789,
 111,
 157,
 508,
 711,
 66,
 117,
 37,
 334,
 107,
 94,
 740,
 514,
 80,
 522,
 89,
 428,
 262,
 112,
 336,
 77,
 162,
 156,
 266,
 82,
 75,
 490,
 68,
 82,
 237,
 79,
 59,
 144,
 290,
 82,
 120,
 120,
 8,
 220,
 16,
 327,
 28,
 92,
 13,
 306,
 136,
 18,
 113,
 453,
 48,
 212,
 402,
 5,
 16,
 69,
 32,
 22,
 8,
 613,
 614,
 40,
 26,
 23,
 613,
 1066,
 12,
 9,
 20,
 20,
 735,
 26,
 13,
 36,
 1347,
 37,
 634,
 32,
 62,
 62,
 42,
 46,
 142,
 60,
 68,
 418,
 28,
 70,
 28,
 10,
 668,
 825,
 775,
 181,
 100,
 63,
 29,
 43,
 48,
 780,
 1504,
 26,
 26,
 24,
 441,
 18,
 38,
 24,
 444,
 444,
 1181,
 1019,
 2077,
 635,
 2114,
 1058,
 1076,
 1076,
 582,
 393,
 158,
 257,
 1820,
 1226,
 1226,
 257,
 187,
 256,
 161,
 161,
 170,
 170,
 373,
 174,
 248,
 248,
 138,
 97,
 24,
 52,
 2165,
 150,
 40,
 422,
 108,
 543,
 85,
 163,
 399,
 346,
 327,
 462,
 611

# 数据清洗
## 数据清洗函数1
- 把的包含在测试集中的起始地和目的地运单号的筛选出来

In [ ]:
def data_process(df, order_list, mode='train'):
    assert mode=='train' or mode=='test'
    if mode == 'train':
        df = df.drop_duplicates(keep='first')
        df = df[df['loadingOrder'].isin(order_list)]
    return df

In [ ]:
def train_process1():
    reader = pd.read_csv("F:/grad/HWC/ChuSai/data/train0523.csv",header =None, names = column_list, iterator=True)
    chunk_cnt = 0
    debug = True
    df = reader.get_chunk(NDATA)
    df = get_data(df, mode='train')
    df = data_process(df, mode='train')
    while chunk_cnt<146:
        try:
            train_data = reader.get_chunk(NDATA)
            train_data = get_data(train_data, mode='train')
            train_data = data_process(train_data, mode='train')
            print('the size is',train_data.shape)
            if train_data.shape[0]>0:
                df = pd.concat([df,train_data])
            else:
                continue
            print('the size is',df.shape,", Finish ",chunk_cnt+1," chunks")
            chunk_cnt+=1
        except:
            break
    # df = df.drop(['Start','End','StartEnd'],axis=1)
    df.to_csv('data/train0628.csv',index = False, header=None, mode='a')
    return df

## 数据清洗函数2
然后是清洗数据(在data_process2()里做的,可以合并到data_process里面)：
- [ ] 卫星或岸基的经纬度数据可能不准，先修正了
- [ ] 速率不符合vesselStatus的(例如'at anchor'的状态速率应该为0)
- [ ] 瞬时速率与相邻索引的时间戳和修正后的经纬度及其一阶变化严重不符的
- [ ] 把eventMMDD.csv的事件和训练集事件联系起来，并筛选行数

## 训练集数据清洗
先是筛选数据到 '/train0621.csv',数据点满足：
- 只留有含出发地目的地的
- 速率不为零
- 速度方向不为-1
- 出发地和目的地元素分别属于，测试集出发地目的地的集合

In [ ]:
def train_process2(order_list):
    # route_files = 'F:/grad/HWC/ChuSai/data/all_routes/'+route_list+'.csv'
    # odf = pd.read_csv(route_files)
    # all_orders = odf['loadingOrder'].unique()
    NDATA = 2000000
    column_list = ['loadingOrder','carrierName','timestamp','longitude',
                  'latitude','vesselMMSI','speed','direction','vesselNextport',
                  'vesselNextportETA','vesselStatus','vesselDatasource','TRANSPORT_TRACE']
    reader = pd.read_csv('F:/grad/HWC/ChuSai/data/train062802.csv',header =None, names = column_list, iterator=True)
    
    order_list = pd.read_csv('F:/grad/HWC/ChuSai/data/all_routess/'+route_list+'.csv')['loadingOrder'].unique()
    chunk_cnt = 0
    debug = True
    df = reader.get_chunk(NDATA)
    df = get_data(df, mode='train')
    df = data_process(df, order_list, mode='train')
    while debug:
        try:
            train_data = reader.get_chunk(NDATA)
            train_data = get_data(train_data, mode='train')
            train_data = data_process(train_data, order_list, mode='train')
            if train_data.shape[0]>0:
                df = pd.concat([df,train_data])
            print('the size is',df.shape,", Finish ",chunk_cnt+2," chunks")
            chunk_cnt+=1

        except:
            break
    # df = df.drop(['Start','End','StartEnd'],axis=1)
    df.sort_values(by=['loadingOrder','timestamp'],inplace=True)
    df.to_csv('F:/grad/HWC/ChuSai/data/train/'+route_list+'.csv', header = None)
    return df

## 第一轮数据清洗

In [ ]:
train_data = train_process1()
train_data.to_csv('data/train0628.csv',index = False, header=None, mode='a')

## 第二轮数据清洗

In [ ]:
for i in range(len(course_list)):
    print(course_list[i])
    route_list = course_list[i]
    
    if not os.path.exists('F:/grad/HWC/ChuSai/data/order/'+route_list+'/'):
        os.mkdir('F:/grad/HWC/ChuSai/data/order/'+route_list+'/')
    train_data = train_process2(route_list)

# 抓取特征
## 特征简介

- 把经纬度和位移转换成xyz坐标和球面最短曲线
- 把船开的方向改成xyz单位向量
- 对于每个运单号，把所有特征的最小值最大值平均数中位数找到

In [ ]:
from math import radians, cos, sin, asin, sqrt
def geodistance(x,lng1,lat1,lng2,lat2):
    #lng1,lat1,lng2,lat2 = (120.12802999999997,30.28708,115.86572000000001,28.7427)
    lng1, lat1, lng2, lat2 = map(radians, [float(x[lng1]), float(x[lat1]), float(x[lng2]), float(x[lat2])]) # 经纬度转换成弧度
    dlon=lng2-lng1
    dlat=lat2-lat1
    a=sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    distance=2*asin(sqrt(a))*6371*1000 # 地球平均半径，6371km # unit: m
    distance=round(distance/1000,3) # unit: km
    return distance

### 训练集分路线特征

In [4]:
# for i in range(len(course_list)):
    #train
    
train_feature_path = 'data/feature/'#+course_list[i]
# print('route name =',course_list[i])

# 搜集特征

train_feature_column = ['loadingOrder','long_diff_from_start','lat_diff_from_start','long_diff_to_end','lat_diff_to_end',\
                   'start_longitude','start_latitude','end_longitude','end_latitude','distance_from_start',\
                   'distance_to_end','speed','direction','time_used']

train_feature = pd.DataFrame(columns=('loadingOrder','total_dist','avg_speed','stop_fraction','label'))
os.chdir('F:/grad/HWC/ChuSai/'+train_feature_path)
train_list = glob.glob('*.csv')
feat = None
for j in range(len(train_list)):
    df_feature = pd.read_csv(train_list[j],index_col=0)
    if df_feature.shape[0]>0:
        feat = pd.concat([feat,df_feature])
        print('j = ',j,',the size of feat is',feat.shape)
    feat.to_csv('F:/grad/HWC/ChuSai/data/trainfeature.csv',index=False)

j =  0 ,the size of feat is (5138, 15)
j =  1 ,the size of feat is (16555, 15)
j =  2 ,the size of feat is (18454, 15)
j =  3 ,the size of feat is (19373, 15)
j =  4 ,the size of feat is (19807, 15)
j =  5 ,the size of feat is (20900, 15)
j =  6 ,the size of feat is (25528, 15)
j =  7 ,the size of feat is (28195, 15)
j =  8 ,the size of feat is (46368, 15)
j =  9 ,the size of feat is (88571, 15)
j =  10 ,the size of feat is (93180, 15)
j =  11 ,the size of feat is (94897, 15)
j =  12 ,the size of feat is (97586, 15)
j =  13 ,the size of feat is (98762, 15)
j =  14 ,the size of feat is (130699, 15)
j =  15 ,the size of feat is (132814, 15)
j =  16 ,the size of feat is (137514, 15)
j =  17 ,the size of feat is (141249, 15)
j =  18 ,the size of feat is (161882, 15)
j =  19 ,the size of feat is (168582, 15)
j =  20 ,the size of feat is (168951, 15)
j =  21 ,the size of feat is (169614, 15)
j =  22 ,the size of feat is (187117, 15)
j =  23 ,the size of feat is (188872, 15)
j =  24 ,the size

- 把训练集数据筛得更适合估算测试集

In [88]:
os.chdir('F:/grad/HWC/ChuSai/data/feature')
train_list = glob.glob('*.csv')
for i in range(len(train_list)):
    print('==========',train_list[i],'started training   ==========')
    train_set = pd.read_csv('F:/grad/HWC/ChuSai/data/feature/'+train_list[i])
    # train_set['time_used'] = np.repeat(0,train_set.shape[0])
    test_set = pd.read_csv('F:/grad/HWC/ChuSai/data/R2_test_featureA.csv')

    train_set['longitude'] = train_set['start_longitude']+train_set['long_diff_from_start']
    test_set['longitude'] = test_set['start_longitude']+test_set['long_diff_from_start']

    print('test_set locations =',(test_set['longitude'].min(),test_set['longitude'].max()))
    if test_set['longitude'].max()-test_set['longitude'].min() > 350:
        train_set['longitude'][train_set['longitude']<0] = train_set['longitude'] + 360
        test_set['longitude'][test_set['longitude']<0] = test_set['longitude'] + 360
    if test_set['longitude'].min()>95:
        train_set = train_set[(train_set['longitude']<test_set['longitude'].max())]
    elif test_set['longitude'].max()<290:
        train_set = train_set[(train_set['longitude']>test_set['longitude'].min())]
    else:
        train_set = train_set[(train_set['longitude']<test_set['longitude'].max()) & (train_set['longitude']>test_set['longitude'].min())]
    train_set = train_set.drop(['longitude'],axis=1)
    train_set.to_csv('F:/grad/HWC/ChuSai/data/feature/'+train_list[i], index = False)

========== AA191175561416.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== AA597788695392.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== AA642567891255.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== AA681773043065.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== AB283635056094.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== AB309086787273.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== AC118422359182.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== AC357556396608.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== AC449966542855.csv started training   ==========
test_set locations = (-178.4

test_set locations = (-178.43339, 179.84885199999997)
========== AX984110307584.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== AY143679328090.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== AY201332896019.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== AY399952630533.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== AZ165684400071.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== AZ276880454886.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== BA256622340782.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== BA520344156377.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== BA553235321543.csv star

test_set locations = (-178.43339, 179.84885199999997)
========== BM957109181718.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== BM993302083237.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== BM997073177986.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== BN331824017295.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== BN415983552324.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== BN653678104459.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== BO245299234486.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== BO309410841219.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== BO427880657831.csv star

test_set locations = (-178.43339, 179.84885199999997)
========== CA314115410850.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== CA403556237677.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== CA481222856960.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== CA819340907501.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== CB197975081208.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== CB317144857784.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== CB367048155632.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== CB458176923066.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== CB599570025427.csv star

test_set locations = (-178.43339, 179.84885199999997)
========== CP344911116512.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== CP472696313592.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== CP635083334107.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== CQ281727211468.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== CQ392659846842.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== CQ512325801120.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== CQ939681686118.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== CR406408585608.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== CR744014965818.csv star

...

test_set locations = (-178.43339, 179.84885199999997)
========== IZ964928738121.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== JA545766488808.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== JB328071757423.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== JB653318560096.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== JC411900252055.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== JC928930232907.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== JD528257243377.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== JD650814796526.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== JF143628642097.csv star

test_set locations = (-178.43339, 179.84885199999997)
========== KC824058368749.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== KD219142085361.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== KD506903072074.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== KD733352981142.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== KE266263106239.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== KE607377663581.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== KE908515928310.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== KF329674265312.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== KF445577824819.csv star

test_set locations = (-178.43339, 179.84885199999997)
========== LG665477429430.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== LG778582236224.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== LG911064678403.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== LH252856548215.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== LH253372634814.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== LH902337014317.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== LH923683357060.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== LI223827960624.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== LI635525050239.csv star

test_set locations = (-178.43339, 179.84885199999997)
========== MK779609573402.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== ML208270379004.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== ML670664516169.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== ML736865855810.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== MM351141639030.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== MM583946867130.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== MN633067748105.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== MO245150925634.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== MO605075930591.csv star

test_set locations = (-178.43339, 179.84885199999997)
========== NM567564337295.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== NM912854190106.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== NN302354859883.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== NN369568639379.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== NN799206924147.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== NN802184540952.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== NO462842002292.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== NO583073595816.csv started training   ==========
test_set locations = (-178.43339, 179.84885199999997)
========== NO820808339376.csv star

- 清洗航程中非停港速度为0的时速，精确到整数kph

In [89]:
os.chdir('F:/grad/HWC/ChuSai/data/feature')
# train_list = glob.glob('*.csv')
for i in range(len(train_list)):
    train_feature_path = 'F:/grad/HWC/ChuSai/data/feature/'+train_list[i]

    # print('route name =',course_list[i])
    train_feature = pd.read_csv(train_feature_path)

    lon_diff = train_feature['long_diff_from_start'].values
    lat_diff = train_feature['lat_diff_from_start'].values
    time_used = train_feature['time_used'].values
    latitude = train_feature['start_latitude'].values + train_feature['lat_diff_from_start'].values
    speed = train_feature['speed'].values
    ind = np.where((time_used > 0) & (speed == 0) & (train_feature['direction'] >= 0))[0]

    speed[ind] = 111.111*3600*(((lon_diff[ind]-lon_diff[ind-1])*np.cos(latitude[ind]*np.pi/180))**2+(lat_diff[ind]-lat_diff[ind-1])**2)**0.5/(time_used[ind]-time_used[ind-1])
    train_feature['speed'] = np.round(speed,0)
    print('i = ',i,', the size is',train_feature.shape)
    train_feature.to_csv('F:/grad/HWC/ChuSai/data/feature/'+train_list[i],index=False)

the size is (5138, 16)
the size is (11417, 16)
the size is (1899, 16)
the size is (919, 16)
the size is (434, 16)
the size is (1093, 16)
the size is (4628, 16)
the size is (2667, 16)
the size is (18173, 16)
the size is (42203, 16)
the size is (4609, 16)
the size is (1717, 16)
the size is (2689, 16)
the size is (1176, 16)
the size is (31937, 16)
the size is (2115, 16)
the size is (4700, 16)
the size is (3735, 16)
the size is (20633, 16)
the size is (6700, 16)
the size is (369, 16)
the size is (663, 16)
the size is (17489, 16)
the size is (1755, 16)
the size is (446, 16)
the size is (4266, 16)
the size is (2697, 16)
the size is (372, 16)
the size is (3465, 16)
the size is (267, 16)
the size is (2071, 16)
the size is (572, 16)
the size is (4565, 16)
the size is (5169, 16)
the size is (3162, 16)
the size is (3316, 16)
the size is (2730, 16)
the size is (370, 16)
the size is (332, 16)
the size is (3426, 16)
the size is (2892, 16)
the size is (3726, 16)
the size is (1833, 16)
the size is (15

the size is (7468, 16)
the size is (11228, 16)
the size is (4690, 16)
the size is (3020, 16)
the size is (1078, 16)
the size is (4115, 16)
the size is (2675, 16)
the size is (1996, 16)
the size is (4946, 16)
the size is (3552, 16)
the size is (7421, 16)
the size is (25422, 16)
the size is (912, 16)
the size is (1761, 16)
the size is (7431, 16)
the size is (2704, 16)
the size is (28670, 16)
the size is (202, 16)
the size is (10602, 16)
the size is (3679, 16)
the size is (1253, 16)
the size is (3380, 16)
the size is (1915, 16)
the size is (6135, 16)
the size is (35367, 16)
the size is (5006, 16)
the size is (1407, 16)
the size is (974, 16)
the size is (4508, 16)
the size is (6329, 16)
the size is (27477, 16)
the size is (16007, 16)
the size is (618, 16)
the size is (4551, 16)
the size is (483, 16)
the size is (2546, 16)
the size is (12050, 16)
the size is (4830, 16)
the size is (1598, 16)
the size is (1181, 16)
the size is (1867, 16)
the size is (4867, 16)
the size is (34424, 16)
the siz

the size is (5297, 16)
the size is (199, 16)
the size is (6819, 16)
the size is (5341, 16)
the size is (20931, 16)
the size is (2591, 16)
the size is (4353, 16)
the size is (1785, 16)
the size is (6411, 16)
the size is (4701, 16)
the size is (228, 16)
the size is (2563, 16)
the size is (14434, 16)
the size is (8087, 16)
the size is (6189, 16)
the size is (1542, 16)
the size is (2319, 16)
the size is (18599, 16)
the size is (13514, 16)
the size is (1377, 16)
the size is (4326, 16)
the size is (15028, 16)
the size is (11110, 16)
the size is (9483, 16)
the size is (1189, 16)
the size is (3842, 16)
the size is (1707, 16)
the size is (4824, 16)
the size is (19441, 16)
the size is (4233, 16)
the size is (4442, 16)
the size is (1131, 16)
the size is (9362, 16)
the size is (3089, 16)
the size is (822, 16)
the size is (7108, 16)
the size is (776, 16)
the size is (5817, 16)
the size is (3489, 16)
the size is (6392, 16)
the size is (6184, 16)
the size is (2782, 16)
the size is (10879, 16)
the siz

the size is (5028, 16)
the size is (395, 16)
the size is (5041, 16)
the size is (14490, 16)
the size is (27987, 16)
the size is (815, 16)
the size is (648, 16)
the size is (29942, 16)
the size is (661, 16)
the size is (6740, 16)
the size is (1844, 16)
the size is (21959, 16)
the size is (3925, 16)
the size is (1702, 16)
the size is (3057, 16)
the size is (3702, 16)
the size is (3665, 16)
the size is (7859, 16)
the size is (221, 16)
the size is (4092, 16)
the size is (39111, 16)
the size is (1495, 16)
the size is (2213, 16)
the size is (5327, 16)
the size is (1686, 16)
the size is (1411, 16)
the size is (22908, 16)
the size is (4495, 16)
the size is (5358, 16)
the size is (680, 16)
the size is (8239, 16)
the size is (34913, 16)
the size is (2867, 16)
the size is (718, 16)
the size is (482, 16)
the size is (1542, 16)
the size is (5388, 16)
the size is (36148, 16)
the size is (554, 16)
the size is (28043, 16)
the size is (18142, 16)
the size is (263, 16)
the size is (10339, 16)
the size i

In [90]:
# for i in range(len(train_list)):
    #train
    
    # train_schedule_path = 'data/schedule/'+course_list[i]
    # train_feature_path = 'data/feature/'+course_list[i]
    # print('route name =',course_list[i])

    # 搜集特征

train_feature_column = ['loadingOrder','long_diff_from_start','lat_diff_from_start','long_diff_to_end','lat_diff_to_end',\
                   'start_longitude','start_latitude','end_longitude','end_latitude','distance_from_start',\
                   'distance_to_end','speed','direction','time_used']

train_feature = pd.DataFrame(columns=('loadingOrder','total_dist','avg_speed','avg_dir','stop_fraction','low_speed_frac',\
                                      'high_speed_frac','label'))
os.chdir('F:/grad/HWC/ChuSai/data/feature')#+train_feature_path)
# train_list = glob.glob('*.csv')
for j in range(len(train_list)):
    df_feature = pd.read_csv(train_list[j])
    df_cuts = df_feature[df_feature['time_used'] == 0]
    loadingOrder = df_feature['loadingOrder'].unique()[0]
    total_dist = df_cuts['distance_to_end'].sum()
    avg_speed = df_feature['speed'][(df_feature['speed']>1) & (df_feature['direction']>=0.)].mean()
    low_speed_frac = df_feature['speed'][(df_feature['speed']>1) & (df_feature['speed']<20) & (df_feature['direction']>=0.)].count()/df_feature.shape[0]
    high_speed_frac = df_feature['speed'][(df_feature['speed']>=20) & (df_feature['direction']>=0.)].count()/df_feature.shape[0]
    avg_dir = df_feature['direction'][df_feature['speed']>=0].mean()
    stop_fraction = df_feature['speed'][df_feature['speed']<=1].count()/df_feature.shape[0]
    total_time = df_cuts['label'].sum()
    train_feature.loc[train_feature.shape[0]] = [loadingOrder,total_dist,avg_speed,avg_dir,stop_fraction,low_speed_frac,\
                                                 high_speed_frac,total_time]

    if j % 20 == 19:
        print(train_feature.shape)
        print('finish ',(j+1),' orders.')
if len(train_list) > 12:
    train_feature = train_feature[train_feature['stop_fraction']<0.33]
    train_feature = train_feature[train_feature['total_dist']<train_feature['total_dist'].quantile(0.95)]
train_feature.to_csv('F:/grad/HWC/ChuSai/data/train_feature.csv',index=False)

(20, 8)
finish  20  orders.
(40, 8)
finish  40  orders.
(60, 8)
finish  60  orders.
(80, 8)
finish  80  orders.
(100, 8)
finish  100  orders.
(120, 8)
finish  120  orders.
(140, 8)
finish  140  orders.
(160, 8)
finish  160  orders.
(180, 8)
finish  180  orders.
(200, 8)
finish  200  orders.
(220, 8)
finish  220  orders.
(240, 8)
finish  240  orders.
(260, 8)
finish  260  orders.
(280, 8)
finish  280  orders.
(300, 8)
finish  300  orders.
(320, 8)
finish  320  orders.
(340, 8)
finish  340  orders.
(360, 8)
finish  360  orders.
(380, 8)
finish  380  orders.
(400, 8)
finish  400  orders.
(420, 8)
finish  420  orders.
(440, 8)
finish  440  orders.
(460, 8)
finish  460  orders.
(480, 8)
finish  480  orders.
(500, 8)
finish  500  orders.
(520, 8)
finish  520  orders.
(540, 8)
finish  540  orders.
(560, 8)
finish  560  orders.
(580, 8)
finish  580  orders.
(600, 8)
finish  600  orders.
(620, 8)
finish  620  orders.
(640, 8)
finish  640  orders.
(660, 8)
finish  660  orders.
(680, 8)
finish  6

In [43]:
train_feature.shape[0]

0

### 测试集分路线特征

In [ ]:
df_test = pd.read_csv('F:/grad/HWC/ChuSai/data/test_featureB.csv')
test_order = df_test['loadingOrder'].unique()

df_test['TRANSPORT_TRACE'] = test_data['TRANSPORT_TRACE']
for i in range(len(course_list)):
    print('route name =',course_list[i])
    temp_test = df_test[df_test['TRANSPORT_TRACE'] == course_list[i]]
    temp_test.to_csv('F:/grad/HWC/ChuSai/data/feature/test/'+course_list[i]+'.csv',index=False)

In [70]:
df_test = pd.read_csv('F:/grad/HWC/ChuSai/data/R2_test_featureA.csv')
test_order = df_test['loadingOrder'].unique()

df_test['TRANSPORT_TRACE'] = test_data['TRANSPORT_TRACE']
# for i in range(len(course_list)):
#     print('route name =',course_list[i])
#     temp_test = df_test[df_test['TRANSPORT_TRACE'] == course_list[i]]
#     dfa=temp_test.groupby('loadingOrder').nth(0).reset_index()
test_order = df_test['loadingOrder'].unique()
test_feature = pd.DataFrame(columns=('loadingOrder','total_dist','avg_speed','avg_dir','stop_fraction','low_speed_frac',\
                                      'high_speed_frac','time_used'))
for j in range(len(test_order)):
    df_vuts = df_test[df_test['loadingOrder'] == test_order[j]]
    loadingOrder = df_vuts['loadingOrder'].unique()[0]
    total_dist = np.sum(df_vuts.iloc[0,9:11])
    avg_speed = df_vuts['speed'][(df_vuts['speed']>1) & (df_vuts['direction']>=0.)].mean()
    low_speed_frac = df_vuts['speed'][(df_vuts['speed']>1) & (df_vuts['speed']<20) & (df_vuts['direction']>=0.)].count()/df_vuts.shape[0]
    high_speed_frac = df_vuts['speed'][(df_vuts['speed']>=20) & (df_vuts['direction']>=0.)].count()/df_vuts.shape[0]
    avg_dir = df_vuts['direction'][df_vuts['direction']>=0.].mean()
    stop_fraction = df_vuts['speed'][df_vuts['speed']<=1].count()/df_vuts.shape[0]
    time_used = df_vuts['time_used'].iloc[-1]
    test_feature.loc[test_feature.shape[0]] = [loadingOrder,total_dist,avg_speed,avg_dir,stop_fraction,low_speed_frac,\
                                                 high_speed_frac,time_used]
    # print(test_feature.loc[test_feature.shape[0]])
    if j%10 == 9:
        print('finish ',(j+1),' orders.')
test_feature.to_csv('F:/grad/HWC/ChuSai/data/test_feature.csv',index=False)

finish  10  orders.
finish  20  orders.
finish  30  orders.
finish  40  orders.
finish  50  orders.
finish  60  orders.
finish  70  orders.
finish  80  orders.
finish  90  orders.
finish  100  orders.
finish  110  orders.
finish  120  orders.
finish  130  orders.
finish  140  orders.
finish  150  orders.
finish  160  orders.
finish  170  orders.
finish  180  orders.
finish  190  orders.
finish  200  orders.
finish  210  orders.


In [69]:
df_vuts

,loadingOrder,long_diff_from_start,lat_diff_from_start,long_diff_to_end,lat_diff_to_end,start_longitude,start_latitude,end_longitude,end_latitude,distance_from_start,distance_to_end,speed,direction,time_used,TRANSPORT_TRACE
24142,VE716598652307,-3.011296,-6.889274,-7.306306,-14.270772,113.863058,22.559462,103.545456,1.399416,828.713,1777.628,0,-1,0.0,CNYTN-ARENA
24143,VE716598652307,-3.578058,-10.116129,-6.739544,-11.043917,113.863058,22.559462,103.545456,1.399416,1186.930,1435.166,0,-1,45430.0,CNYTN-ARENA
24144,VE716598652307,-4.684725,-13.877795,-5.632877,-7.282251,113.863058,22.559462,103.545456,1.399416,1622.169,1021.977,31,21600,94220.0,CNYTN-ARENA
24145,VE716598652307,-4.940941,-14.222529,-5.376661,-6.937517,113.863058,22.559462,103.545456,1.399416,1667.244,974.422,0,-1,100170.0,CNYTN-ARENA
24146,VE716598652307,-9.018546,-20.352132,-1.299056,-0.807914,113.863058,22.559462,103.545456,1.399416,2463.559,170.044,23,20200,202460.0,CNYTN-ARENA
24147,VE716598652307,-9.039355,-20.403665,-1.278247,-0.756381,113.863058,22.559462,103.545456,1.399416,2469.738,165.095,23,20230,203400.0,CNYTN-ARENA
24148,VE716598652307,-9.046415,-20.421175,-1.271187,-0.738871,113.863058,22.559462,103.545456,1.399416,2471.837,163.433,23,20200,203720.0,CNYTN-ARENA
24149,VE716598652307,-9.049471,-20.428805,-1.268131,-0.731241,113.863058,22.559462,103.545456,1.399416,2472.751,162.714,23,20190,203860.0,CNYTN-ARENA


# 训练模型
## LightGBM算法

- 只需要调params和clf = lgb.train()里面的参数

In [91]:
def build_model(train, test, n_splits, pred, residual_list, seed=1080, is_shuffle=True):
    train_pred = np.zeros((train.shape[0], ))
    test_pred = np.zeros((test.shape[0], ))
    # Kfold
    fold = KFold(n_splits=n_splits, shuffle=is_shuffle, random_state=seed)
    kf_way = fold.split(train[pred])
    # params
    params = {
        'learning_rate': 0.05,
        'boosting_type': 'gbdt',
        'objective': 'regression_l2',
        'num_leaves': 31,
        'metric': 'mse',
        'feature_fraction': 0.6,
        'bagging_fraction': 0.7,
        'bagging_freq': 6,
        'seed': 2,
        'random_state': 32,
        'lambda_l2': 0.5,
        'bagging_seed': 1,
        'feature_fraction_seed': 7,
        'min_data_in_leaf': 15,
        'nthread': 8,
        'verbose': 1,
    }
    # train
    for n_fold, (train_idx, valid_idx) in enumerate(kf_way, start=1):
        train_x, train_y = train[pred].iloc[train_idx], train[residual_list[-2]].iloc[train_idx]
        valid_x, valid_y = train[pred].iloc[valid_idx], train[residual_list[-2]].iloc[valid_idx]
        # 数据加载
        n_train = lgb.Dataset(train_x, label=train_y)
        n_valid = lgb.Dataset(valid_x, label=valid_y)

        clf = lgb.train(
            params=params,
            train_set=n_train,
            num_boost_round=20000,
            valid_sets=[n_valid],
            early_stopping_rounds=100,
            verbose_eval=100
        )
        

        train_pred[valid_idx] = clf.predict(valid_x, num_iteration=clf.best_iteration)
        test_pred += clf.predict(test[pred], num_iteration=clf.best_iteration)/fold.n_splits
    train[residual_list[-1]] = train_pred-train[residual_list[-2]].values
    
    test[residual_list[-2]] = test_pred
    
    return test[residual_list[0:-1]], train

## 按路线肢解训练大法
- 把测试集按路线肢解以后(其实早在处理训练集的时候就有把测试集肢解的打算的)
- 迭代训练，每次训练出来result和train这两个表格就多一列nj级迭代残差，nj=12
- 使用nk个种子，nk增长方式按路线内运单号数量的三倍递增，12封顶
- 比较各个迭代残差的方差标准差，从一级残差加到方差加到n_max级残差(也就是标准差最小的一级)
- 把迭代残差和测试集预测值加起来，就得最后秒数预测结果
- 取nk个种子的预测结果的80-95百分位，以避免outliers
- 先把res1(结果)按照订单号字母顺序排列，然后把ETA按测试集B榜每个运单号的列数做个重复，加上Onboarddate就可以放在test_data并且整理出results
- [ ] 如果要改进算法的话，把加权平均考虑进去就可以了

In [93]:
res = None
# for i in range(len(course_list)):
# print('==========',course_list[i],'started training   ==========')
train_set = pd.read_csv('F:/grad/HWC/ChuSai/data/train_feature.csv')
# train_set['time_used'] = np.repeat(0,train_set.shape[0])
test_set = pd.read_csv('F:/grad/HWC/ChuSai/data/test_feature.csv')
train_set['label'] = train_set['label']*3600
# print("reading the route:", course_list[i],',the size of the test =',test_set.shape)

# ETA = np.nan*np.zeros((test_set.shape[0],21*4))
train_orders = train_set['loadingOrder'].unique()

nk, nj = min(2 * len(train_orders) + 1, 8), 12
ETA = np.nan*np.zeros((test_set.shape[0],nk))
# j = 1
# print('========== train by',byorder,'orders ============')
# torder = train_orders[byorder*j:min(byorder*(j+1),len(train_orders))]
# train_sett = train_set[train_set['loadingOrder'].isin(torder)]

for k in range(nk):
    residual_list = ['loadingOrder','label']
    for j in range(nj):
        residual = 'residual_'+str(j+1)
        residual_list.append(residual)
        features = [c for c in train_set.columns if c not in residual_list]
        result, train_set = build_model(train_set, test_set, np.min([4*train_set.shape[0],2*test_set.shape[0],25]), features, residual_list, seed=1080+7*k, is_shuffle=True)
        maxmin = np.array([train_set[residual_list[-1]].max(),train_set[residual_list[-1]].min()])
        zscore = (maxmin - train_set[residual_list[-1]].mean())/train_set[residual_list[-1]].std()
        print('==========j =',j,',k =',k,'the zscore is',zscore,'==========')
        print('the size of train is ',train_set.shape)
        while (train_set.shape[0] >= 10) and ((zscore[0] > 3.) or (zscore[1] < -3.)):
            if zscore[0] > 3.:
                train_set = train_set[~(train_set[residual_list[-1]] == maxmin[0])]
            if zscore[1] < -3.:
                train_set = train_set[~(train_set[residual_list[-1]] == maxmin[1])]
            maxmin = np.array([train_set[residual_list[-1]].max(),train_set[residual_list[-1]].min()])
            zscore = (maxmin - train_set[residual_list[-1]].mean())/train_set[residual_list[-1]].std()
    print(train_set[residual_list[2::]])
    train_set = train_set.drop(residual_list[2::],axis=1)
    result_std = result.std(axis=1)
    if np.shape(np.where(result.columns[1::]==result[1::].std().idxmin(axis=1))[0])[0] > 0:
        res_ind = np.where(result.columns[1::]==result.std().idxmin(axis=1))[0][0]
    else:
        res_ind = 4
    # print('==========',result[1::].std().idxmin(axis=1),',',res_ind,'has the smallest standard deviation within the route',course_list[i],'==========')
    ETA[:,k] = test_set['time_used'] + result[residual_list[1:res_ind+2]].sum(axis=1) 
ETA_10 = np.percentile(ETA,10,axis=1)
print(ETA_10)
result['label'] = ETA_10
# res = pd.concat([res,result])
# print('==========',course_list[i],'completed training ==========')

Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 6.46685e+10
[200]	valid_0's l2: 5.74113e+10
[300]	valid_0's l2: 5.67718e+10
Early stopping, best iteration is:
[213]	valid_0's l2: 5.61871e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 1.13155e+11
[200]	valid_0's l2: 9.42732e+10
[300]	valid_0's l2: 9.44991e+10
[400]	valid_0's l2: 9.08288e+10
[500]	valid_0's l2: 8.84032e+10
[600]	valid_0's l2: 8.6212e+10
[700]	valid_0's l2: 8.57506e+10
[800]	valid_0's l2: 8.57613e+10
Early stopping, best iteration is:
[763]	valid_0's l2: 8.53681e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 3.59676e+10
[200]	valid_0's l2: 3.29776e+10
[300]	valid_0's l2: 3.24238e+10
Early stopping, best iteration is:
[244]	valid_0's l2: 3.08994e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 8.2174e+10
[200]	valid_0's l2: 6.68225e+10
[300]	valid_0's l2: 6.18838e+10
[400]	val

[100]	valid_0's l2: 1.71706e+10
Early stopping, best iteration is:
[6]	valid_0's l2: 1.3834e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 2.61839e+10
[200]	valid_0's l2: 2.82954e+10
Early stopping, best iteration is:
[113]	valid_0's l2: 2.53083e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 2.68404e+10
Early stopping, best iteration is:
[7]	valid_0's l2: 2.47092e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 1.70632e+10
Early stopping, best iteration is:
[19]	valid_0's l2: 1.54624e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 2.79488e+10
Early stopping, best iteration is:
[6]	valid_0's l2: 2.47097e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 1.7688e+10
[200]	valid_0's l2: 1.78105e+10
Early stopping, best iteration is:
[120]	valid_0's l2: 1.72097e+10
Training until validation scores don't im

[100]	valid_0's l2: 2.33229e+10
Early stopping, best iteration is:
[1]	valid_0's l2: 2.02632e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 2.17069e+10
Early stopping, best iteration is:
[5]	valid_0's l2: 1.75908e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 1.48312e+10
Early stopping, best iteration is:
[2]	valid_0's l2: 1.12295e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 1.72506e+10
Early stopping, best iteration is:
[6]	valid_0's l2: 1.55076e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 1.16499e+10
Early stopping, best iteration is:
[1]	valid_0's l2: 9.56606e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 1.30801e+10
Early stopping, best iteration is:
[19]	valid_0's l2: 1.02182e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 3.00139e+10
Early stoppin

[100]	valid_0's l2: 1.59058e+10
Early stopping, best iteration is:
[1]	valid_0's l2: 1.28269e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 1.66695e+10
Early stopping, best iteration is:
[1]	valid_0's l2: 1.27168e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 2.73204e+10
Early stopping, best iteration is:
[1]	valid_0's l2: 2.59416e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 1.68817e+10
Early stopping, best iteration is:
[12]	valid_0's l2: 1.3662e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 1.30708e+10
[200]	valid_0's l2: 1.34275e+10
Early stopping, best iteration is:
[129]	valid_0's l2: 1.21521e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 2.58684e+10
Early stopping, best iteration is:
[1]	valid_0's l2: 2.07319e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_

[100]	valid_0's l2: 1.70093e+10
Early stopping, best iteration is:
[1]	valid_0's l2: 1.2863e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 2.99106e+10
Early stopping, best iteration is:
[1]	valid_0's l2: 2.65415e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 1.74408e+10
Early stopping, best iteration is:
[6]	valid_0's l2: 1.35395e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 1.52245e+10
Early stopping, best iteration is:
[1]	valid_0's l2: 1.23078e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 2.66485e+10
Early stopping, best iteration is:
[1]	valid_0's l2: 2.08409e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 1.73506e+10
Early stopping, best iteration is:
[1]	valid_0's l2: 1.23762e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 1.45347e+10
Early stopping,

...

[100]	valid_0's l2: 7.78686e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 6.59277e+09
==========j = 1 ,k = 7 the zscore is [ 3.0281438  -2.97468598] ==========
the size of train is  (878, 10)
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 6.05852e+09
Early stopping, best iteration is:
[3]	valid_0's l2: 5.36432e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 7.00676e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 5.72943e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 9.68482e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 6.97884e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 5.451e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 3.9826e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 3.46061e+09
Early stopping, best iteration is:
[12]	valid_0's l2: 3.01335e+09
Trai

[100]	valid_0's l2: 6.6057e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 5.20874e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 6.76011e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 5.51229e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 5.12574e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 4.42417e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 3.2226e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 2.3178e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 4.66928e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 3.34896e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 6.20485e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 4.79916e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 8.77264e+09
Early stopping, b

[100]	valid_0's l2: 7.09626e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 5.49324e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 5.24869e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 4.49553e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 2.97929e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 2.38926e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 4.77544e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 3.33717e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 6.36639e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 4.77531e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 9.09044e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 6.38232e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 5.78199e+09
Early stopping

[100]	valid_0's l2: 9.29486e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 7.6754e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 5.41675e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 4.07016e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 2.74962e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 2.23629e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 9.16038e+09
Early stopping, best iteration is:
[2]	valid_0's l2: 6.55934e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 8.89466e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 6.54327e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 1.01087e+10
Early stopping, best iteration is:
[1]	valid_0's l2: 7.95653e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 6.15401e+09
Early stopping,

[100]	valid_0's l2: 5.32012e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 4.076e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 2.92628e+09
Early stopping, best iteration is:
[2]	valid_0's l2: 2.3235e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 9.71383e+09
Early stopping, best iteration is:
[2]	valid_0's l2: 6.6559e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 9.53491e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 6.58726e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 1.01227e+10
Early stopping, best iteration is:
[1]	valid_0's l2: 8.08843e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 6.08677e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 5.10015e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 8.7565e+09
Early stopping, bes

In [94]:
result

,loadingOrder,label,residual_1,residual_2,residual_3,residual_4,residual_5,residual_6,residual_7,residual_8,residual_9,residual_10,residual_11
0,QM149151037282,1.257861e+06,143.843211,605.070534,-711.054709,194.190760,-487.716398,374.743030,-341.970919,83.769637,-408.951042,652.217406,-688.762528
1,WA967563758558,2.144357e+06,-3399.530384,3051.721976,-2940.928579,2299.362089,-2057.402155,1643.667180,-1498.008060,981.610575,-1210.611341,1535.593699,-1660.267203
2,DS668820049737,2.460507e+06,-1655.949480,2112.134690,-1735.693398,1726.066823,-1802.551493,1495.250385,-1264.242017,864.303388,-1058.593898,1478.966935,-1315.890695
3,TO631302419282,2.788956e+06,-1288.478267,2799.077947,-2462.141581,1658.647823,-1725.143371,1406.806732,-1259.929520,788.281746,-860.760884,1239.296852,-1321.831956
4,WF474688448184,2.296243e+06,-569.632880,2163.409899,-2132.103881,1550.647073,-1774.958613,1346.149484,-1305.704374,225.771825,-506.261943,1057.907199,-1236.193857
...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,OZ881442320812,6.377030e+05,-991.504650,2232.410540,-1120.349511,1872.805068,-1533.437050,1125.886801,-1232.321411,988.920437,-1038.171868,961.152924,-49.757104
215,YR704016629917,3.170543e+05,8492.417206,1029.819122,249.404177,1190.063251,-616.969170,403.937142,-562.948336,674.180374,-700.964400,491.787672,182.320609
216,RU337264157954,5.210658e+05,-4186.460487,2640.783165,-2878.146589,1788.035360,-1301.391618,825.758118,-900.640485,850.173928,-761.526936,619.168295,-811.434235
217,VZ162290304055,4.469366e+05,5329.155504,1312.723089,-968.840012,1317.381027,-734.573470,543.235389,-562.948336,360.642255,-498.900466,374.066516,-411.552135


In [97]:
res1 = result.sort_values(by='loadingOrder')
ETA_replace = []
for j in range(order_list.shape[0]):
    ETA_replace.extend(list(np.repeat(res1.iloc[j,1],order_length[j])))
test_data = test_data.sort_values(by = ['loadingOrder'])


In [98]:
test_data['ETA'] = ETA_replace
test_data['ETA'] = (test_data['onboardDate'] + test_data['ETA'].apply(lambda x:pd.Timedelta(seconds=x))).apply(lambda x:x.strftime('%Y/%m/%d  %H:%M:%S'))

- BP订单到了靠近新加坡的MYTPP港口以后还在继续往西走，并且订单号还在延续，可能为意外，因此计算ETA按照到达MYPKG的时间来算
- 打开谷歌地球量路程，最后一个点到(吉隆坡附近的)MYPKG的距离大约是40km，并且假定平均时速仅为20多kmh，还需要花90分钟左右
- 并且最后一次追踪时间为2020/03/15  12:28:xx，因此预估到达时间为 (瞎猜的)

In [ ]:
test_data['ETA'][test_data['loadingOrder'] == order_list[i]]

In [ ]:
test_data['ETA'][test_data['loadingOrder'] == order_list[3]] = pd.datetime(2020,3,16,8,13,15).strftime('%Y/%m/%d  %H:%M:%S')

In [99]:
test_data.drop(['direction','TRANSPORT_TRACE'],axis=1,inplace=True)
test_data['onboardDate'] = test_data['onboardDate'].apply(lambda x:x.strftime('%Y/%m/%d  %H:%M:%S'))
test_data['creatDate'] = pd.datetime.now().strftime('%Y/%m/%d  %H:%M:%S')
test_data['timestamp'] = timestamp.astype(str)
# 整理columns顺序
results = test_data[['loadingOrder', 'timestamp', 'longitude', 'latitude', 'carrierName', 'vesselMMSI', 'onboardDate', 'ETA', 'creatDate']]
results = results.sort_values(by = ['timestamp'])
results.to_csv('F:/grad/HWC/ChuSai/result071302.csv', index=False)

# MSE评分（后记）
- 现在已经2382.19分，截至2020/06/29  18:20:02，咱们排名是第78名，继续上分还得靠各位改进特征了